### Stock Return Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime

import torch

torch.manual_seed(0)
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
stock_ticker_list = pd.read_csv('/content/drive/MyDrive/WorkSpace/COMP3027J/data/Stock/Stock_Ticker_List.csv')

In [4]:
stock_ticker_list.head()

,Symbol,Name,Sector
0,AAPL,Apple Inc.,Information Technology
1,ABBV,AbbVie Inc.,Health Care
2,ABT,Abbott Laboratories,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe Inc.,Information Technology


#### Read one asset price

In [5]:
aapl_df = pd.read_csv('/content/drive/MyDrive/WorkSpace/COMP3027J/data/Stock/Pretest_data/AAPL.csv')

In [6]:
aapl_df.head(16)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-04-28,12.363929,12.491071,12.340000,12.383929,10.667065,360959200
1,2011-04-29,12.385000,12.641071,12.381071,12.504643,10.771044,1006345200
2,2011-05-02,12.490714,12.516786,12.339286,12.367143,10.652606,442713600
3,2011-05-03,12.428214,12.496071,12.343571,12.435714,10.711670,313348000
4,2011-05-04,12.437857,12.565357,12.388571,12.484643,10.753815,389250400
5,2011-05-05,12.442857,12.533929,12.358929,12.383929,10.667065,335969200
6,2011-05-06,12.488929,12.500000,12.364643,12.380714,10.664294,280134400
7,2011-05-09,12.423571,12.471429,12.376071,12.414286,10.693214,204747200
8,2011-05-10,12.460357,12.488929,12.380714,12.480357,10.750122,282091600
9,2011-05-11,12.465000,12.500000,12.330000,12.401071,10.681831,336000000


In [7]:
len(aapl_df)

2514

In [8]:
aapl_df['Date'] = pd.to_datetime(aapl_df['Date'])
aapl_df['day_of_week'] = aapl_df['Date'].dt.day_name()
aapl_df.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume,day_of_week
2504,2021-04-12,132.520004,132.850006,130.630005,131.240005,131.240005,91420000,Monday
2505,2021-04-13,132.440002,134.660004,131.929993,134.429993,134.429993,91266500,Tuesday
2506,2021-04-14,134.940002,135.000000,131.660004,132.029999,132.029999,87222800,Wednesday
2507,2021-04-15,133.820007,135.000000,133.639999,134.500000,134.500000,89347100,Thursday
2508,2021-04-16,134.300003,134.669998,133.279999,134.160004,134.160004,84818500,Friday
2509,2021-04-19,133.509995,135.470001,133.339996,134.839996,134.839996,94264200,Monday
2510,2021-04-20,135.020004,135.529999,131.809998,133.110001,133.110001,94812300,Tuesday
2511,2021-04-21,132.360001,133.750000,131.300003,133.500000,133.500000,68847100,Wednesday
2512,2021-04-22,133.039993,134.149994,131.410004,131.940002,131.940002,84566500,Thursday
2513,2021-04-23,132.160004,135.119995,132.160004,134.320007,134.320007,78657500,Friday


In [9]:
columns = ['Open','High','Low','Close']

In [10]:
aapl_df[columns].iloc[1:]

,Open,High,Low,Close
1,12.385000,12.641071,12.381071,12.504643
2,12.490714,12.516786,12.339286,12.367143
3,12.428214,12.496071,12.343571,12.435714
4,12.437857,12.565357,12.388571,12.484643
5,12.442857,12.533929,12.358929,12.383929
...,...,...,...,...
2509,133.509995,135.470001,133.339996,134.839996
2510,135.020004,135.529999,131.809998,133.110001
2511,132.360001,133.750000,131.300003,133.500000
2512,133.039993,134.149994,131.410004,131.940002


In [11]:
aapl_df[columns].iloc[:-1]

,Open,High,Low,Close
0,12.363929,12.491071,12.340000,12.383929
1,12.385000,12.641071,12.381071,12.504643
2,12.490714,12.516786,12.339286,12.367143
3,12.428214,12.496071,12.343571,12.435714
4,12.437857,12.565357,12.388571,12.484643
...,...,...,...,...
2508,134.300003,134.669998,133.279999,134.160004
2509,133.509995,135.470001,133.339996,134.839996
2510,135.020004,135.529999,131.809998,133.110001
2511,132.360001,133.750000,131.300003,133.500000


In [12]:
a_df2 = aapl_df[columns].iloc[1:].values/aapl_df[columns].iloc[:-1] -1
a_df2.head(15)

,Open,High,Low,Close
0,0.001704,0.012009,0.003328,0.009748
1,0.008536,-0.009832,-0.003375,-0.010996
2,-0.005004,-0.001655,0.000347,0.005545
3,0.000776,0.005545,0.003646,0.003935
4,0.000402,-0.002501,-0.002393,-0.008067
5,0.003703,-0.002707,0.000462,-0.000260
6,-0.005233,-0.002286,0.000924,0.002712
7,0.002961,0.001403,0.000375,0.005322
8,0.000373,0.000886,-0.004096,-0.006353
9,-0.008309,-0.008229,-0.008603,-0.001901


In [13]:
a_df2 = a_df2*100
a_df2   

,Open,High,Low,Close
0,0.170427,1.200862,0.332828,0.974765
1,0.853564,-0.983190,-0.337493,-1.099598
2,-0.500372,-0.165496,0.034725,0.554462
3,0.077586,0.554465,0.364563,0.393457
4,0.040201,-0.250119,-0.239270,-0.806701
...,...,...,...,...
2508,-0.588242,0.594047,0.045016,0.506852
2509,1.131009,0.044288,-1.147442,-1.282999
2510,-1.970081,-1.313361,-0.386916,0.292990
2511,0.513745,0.299061,0.083778,-1.168537


In [14]:
pct_change = aapl_df[columns].pct_change()
pct_change

,Open,High,Low,Close
0,NaN,NaN,NaN,NaN
1,0.001704,0.012009,0.003328,0.009748
2,0.008536,-0.009832,-0.003375,-0.010996
3,-0.005004,-0.001655,0.000347,0.005545
4,0.000776,0.005545,0.003646,0.003935
...,...,...,...,...
2509,-0.005882,0.005940,0.000450,0.005069
2510,0.011310,0.000443,-0.011474,-0.012830
2511,-0.019701,-0.013134,-0.003869,0.002930
2512,0.005137,0.002991,0.000838,-0.011685


In [15]:
b_df = a_df2.shift(-1)
b_df = b_df.rename({'Open':'Open+1','High':'High+1','Low':'Low+1','Close':'Close+1','Volume':'Volume+1'},axis=1)


In [16]:
ab_df = pd.concat([a_df2, b_df], axis=1)
ab_df

,Open,High,Low,Close,Open+1,High+1,Low+1,Close+1
0,0.170427,1.200862,0.332828,0.974765,0.853564,-0.983190,-0.337493,-1.099598
1,0.853564,-0.983190,-0.337493,-1.099598,-0.500372,-0.165496,0.034725,0.554462
2,-0.500372,-0.165496,0.034725,0.554462,0.077586,0.554465,0.364563,0.393457
3,0.077586,0.554465,0.364563,0.393457,0.040201,-0.250119,-0.239270,-0.806701
4,0.040201,-0.250119,-0.239270,-0.806701,0.370269,-0.270696,0.046237,-0.025960
...,...,...,...,...,...,...,...,...
2508,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999
2509,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990
2510,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537
2511,0.513745,0.299061,0.083778,-1.168537,-0.661447,0.723072,0.570733,1.803854


In [17]:
def shift_concat_df(input_df, sequence_length, rename_type=1):
    a_df = input_df.copy()
    for i in range(sequence_length-1):
        b_df = input_df.shift(-1*(i+1))
        if rename_type is 1:
            b_df = b_df.rename({'Open':'Open+'+str(i+1),'High':'High+'+str(i+1),'Low':'Low+'+str(i+1),'Close':'Close+'+str(i+1),'Volume':'Volume+'+str(i+1)},axis=1)
        else:
            b_df = b_df.rename({'Close':'Close+'+str(i+1)},axis=1)
        a_df = pd.concat([a_df, b_df], axis=1)
    return a_df

In [18]:
X = shift_concat_df(a_df2, 10)
X.tail(15)

,Open,High,Low,Close,Open+1,High+1,Low+1,Close+1,Open+2,High+2,Low+2,Close+2,Open+3,High+3,Low+3,Close+3,Open+4,High+4,Low+4,Close+4,Open+5,High+5,Low+5,Close+5,Open+6,High+6,Low+6,Close+6,Open+7,High+7,Low+7,Close+7,Open+8,High+8,Low+8,Close+8,Open+9,High+9,Low+9,Close+9
2498,0.169820,1.594462,0.473510,2.357725,2.123191,0.768860,2.096369,0.246225,-0.529643,0.621412,-0.405891,1.339040,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785
2499,2.123191,0.768860,2.096369,0.246225,-0.529643,0.621412,-0.405891,1.339040,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852
2500,-0.529643,0.621412,-0.405891,1.339040,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999
2501,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990
2502,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537
2503,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537,-0.661447,0.723072,0.570733,1.803854
2504,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537,-0.661447,0.723072,0.570733,1.803854,NaN,NaN,NaN,NaN
2505,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537,-0.661447,0.723072,0.570733,1.803854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2506,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537,-0.661447,0.723072,0.570733,1.803854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2507,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537,-0.661447,0.723072,0.570733,1.803854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
len(X)

2513

In [20]:
X = X.loc[0:len(X)-10]
X.tail()

,Open,High,Low,Close,Open+1,High+1,Low+1,Close+1,Open+2,High+2,Low+2,Close+2,Open+3,High+3,Low+3,Close+3,Open+4,High+4,Low+4,Close+4,Open+5,High+5,Low+5,Close+5,Open+6,High+6,Low+6,Close+6,Open+7,High+7,Low+7,Close+7,Open+8,High+8,Low+8,Close+8,Open+9,High+9,Low+9,Close+9
2499,2.123191,0.768860,2.096369,0.246225,-0.529643,0.621412,-0.405891,1.339040,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852
2500,-0.529643,0.621412,-0.405891,1.339040,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999
2501,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990
2502,0.659175,2.032360,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537
2503,2.095532,-0.142805,0.895963,-1.323304,-0.060370,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.000000,1.503870,1.870788,0.358688,-0.244446,-0.269381,-0.252785,-0.588242,0.594047,0.045016,0.506852,1.131009,0.044288,-1.147442,-1.282999,-1.970081,-1.313361,-0.386916,0.292990,0.513745,0.299061,0.083778,-1.168537,-0.661447,0.723072,0.570733,1.803854


In [21]:
len(X)

2504

In [22]:
X_train = X.loc[0:len(X)-11]
X_train.tail()

,Open,High,Low,Close,Open+1,High+1,Low+1,Close+1,Open+2,High+2,Low+2,Close+2,Open+3,High+3,Low+3,Close+3,Open+4,High+4,Low+4,Close+4,Open+5,High+5,Low+5,Close+5,Open+6,High+6,Low+6,Close+6,Open+7,High+7,Low+7,Close+7,Open+8,High+8,Low+8,Close+8,Open+9,High+9,Low+9,Close+9
2489,0.358632,2.009390,0.484627,2.833571,2.493144,0.298696,1.563277,-0.688871,-0.413526,-1.078555,-1.694776,-1.999351,-2.670574,-1.008949,-0.891147,0.416354,0.677595,-0.147954,-0.067228,0.514141,1.080185,0.905498,1.522036,0.148503,-1.265928,-1.778431,-1.548913,-1.227447,1.282159,2.591358,1.926637,1.876564,1.652283,0.534329,1.106064,0.695864,0.169820,1.594462,0.473510,2.357725
2490,2.493144,0.298696,1.563277,-0.688871,-0.413526,-1.078555,-1.694776,-1.999351,-2.670574,-1.008949,-0.891147,0.416354,0.677595,-0.147954,-0.067228,0.514141,1.080185,0.905498,1.522036,0.148503,-1.265928,-1.778431,-1.548913,-1.227447,1.282159,2.591358,1.926637,1.876564,1.652283,0.534329,1.106064,0.695864,0.169820,1.594462,0.473510,2.357725,2.123191,0.768860,2.096369,0.246225
2491,-0.413526,-1.078555,-1.694776,-1.999351,-2.670574,-1.008949,-0.891147,0.416354,0.677595,-0.147954,-0.067228,0.514141,1.080185,0.905498,1.522036,0.148503,-1.265928,-1.778431,-1.548913,-1.227447,1.282159,2.591358,1.926637,1.876564,1.652283,0.534329,1.106064,0.695864,0.169820,1.594462,0.473510,2.357725,2.123191,0.768860,2.096369,0.246225,-0.529643,0.621412,-0.405891,1.339040
2492,-2.670574,-1.008949,-0.891147,0.416354,0.677595,-0.147954,-0.067228,0.514141,1.080185,0.905498,1.522036,0.148503,-1.265928,-1.778431,-1.548913,-1.227447,1.282159,2.591358,1.926637,1.876564,1.652283,0.534329,1.106064,0.695864,0.169820,1.594462,0.473510,2.357725,2.123191,0.768860,2.096369,0.246225,-0.529643,0.621412,-0.405891,1.339040,2.479532,1.930895,2.700979,1.923377
2493,0.677595,-0.147954,-0.067228,0.514141,1.080185,0.905498,1.522036,0.148503,-1.265928,-1.778431,-1.548913,-1.227447,1.282159,2.591358,1.926637,1.876564,1.652283,0.534329,1.106064,0.695864,0.169820,1.594462,0.473510,2.357725,2.123191,0.768860,2.096369,0.246225,-0.529643,0.621412,-0.405891,1.339040,2.479532,1.930895,2.700979,1.923377,0.659175,2.032360,0.739182,2.025161


In [23]:
len(X_train)

2494

In [24]:
X_test = X.loc[len(X_train)+4:len(X_train)+4]
X_test

,Open,High,Low,Close,Open+1,High+1,Low+1,Close+1,Open+2,High+2,Low+2,Close+2,Open+3,High+3,Low+3,Close+3,Open+4,High+4,Low+4,Close+4,Open+5,High+5,Low+5,Close+5,Open+6,High+6,Low+6,Close+6,Open+7,High+7,Low+7,Close+7,Open+8,High+8,Low+8,Close+8,Open+9,High+9,Low+9,Close+9
2498,0.16982,1.594462,0.47351,2.357725,2.123191,0.76886,2.096369,0.246225,-0.529643,0.621412,-0.405891,1.33904,2.479532,1.930895,2.700979,1.923377,0.659175,2.03236,0.739182,2.025161,2.095532,-0.142805,0.895963,-1.323304,-0.06037,1.362437,0.995168,2.430652,1.887647,0.252485,-0.204646,-1.785311,-0.829995,0.0,1.50387,1.870788,0.358688,-0.244446,-0.269381,-0.252785


In [25]:
a_df3 = a_df2[['Close']]
Y_df = shift_concat_df(a_df3.shift(-10),5,2)
Y_df

,Close,Close+1,Close+2,Close+3,Close+4
0,-1.751448,-2.114539,0.852089,1.109654,0.194196
1,-2.114539,0.852089,1.109654,0.194196,-1.559334
2,0.852089,1.109654,0.194196,-1.559334,-0.244621
3,1.109654,0.194196,-1.559334,-0.244621,-0.660880
4,0.194196,-1.559334,-0.244621,-0.660880,1.381735
...,...,...,...,...,...
2508,NaN,NaN,NaN,NaN,NaN
2509,NaN,NaN,NaN,NaN,NaN
2510,NaN,NaN,NaN,NaN,NaN
2511,NaN,NaN,NaN,NaN,NaN


In [26]:
def add_5day_return(df):
    y3 = df/100+1
    y3 = (y3.prod(axis = 1)-1)*100
    df['5-days']=y3
    return df

In [27]:
Y_train = Y_df.loc[0:len(X_train)-1]
Y_train = add_5day_return(Y_train)
Y_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Close,Close+1,Close+2,Close+3,Close+4,5-days
0,-1.751448,-2.114539,0.852089,1.109654,0.194196,-1.742787
1,-2.114539,0.852089,1.109654,0.194196,-1.559334,-1.550657
2,0.852089,1.109654,0.194196,-1.559334,-0.244621,0.330033
3,1.109654,0.194196,-1.559334,-0.244621,-0.660880,-1.175105
4,0.194196,-1.559334,-0.244621,-0.660880,1.381735,-0.909172
...,...,...,...,...,...,...
2489,0.246225,1.339040,1.923377,2.025161,-1.323304,4.241465
2490,1.339040,1.923377,2.025161,-1.323304,2.430652,6.512950
2491,1.923377,2.025161,-1.323304,2.430652,-1.785311,3.229083
2492,2.025161,-1.323304,2.430652,-1.785311,1.870788,3.175820


In [28]:

Y_test = Y_df.loc[len(X_train)+4:len(X_train)+4]
Y_test = add_5day_return(Y_test)
Y_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Close,Close+1,Close+2,Close+3,Close+4,5-days
2498,0.506852,-1.282999,0.29299,-1.168537,1.803854,0.119263


In [29]:
from torch.utils.data import Dataset
class stock_dataset(Dataset):
    def __init__(self, input_df, label_df, transform=None):
        self.len = len(input_df)
        self.X = torch.from_numpy(input_df.values).reshape(len(input_df),10,4)
        self.Y = torch.from_numpy(label_df.values)
        self.transform = transform
    
    def __getitem__(self, index):
        sample=self.X[index], self.Y[index]
        if self.transform:
            sample.self.transform(sample)
        return sample
    
    def __len__(self):
        return self.len

In [30]:
import torch
import torch.nn as nn

In [31]:
# If there's a GPU available...
if torch.cuda.is_available():    
    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [63]:
# How many epoch we want to use
num_epochs = 100
# Learning rate
learning_rate = 0.001
# Input size (how many sequence)
input_size = 4
# Each sequence length
sequence_length = 10
# Hidden size - number of features of the hidden state for RNN
hidden_size = 60
# Number of layers
num_layers = 1
# Number of classes
num_predictions = 6
# The number of samples selected for one training session.
batch_size = 64

In [34]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout = 0.5, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))    # _ : (hn, cn)
        # out: batch_size, seq_length, hidden_size
        
        out = out[:, -1, :]   # only get last sequence
        
        #out = self.fc(lstm_out.view(-1,hidden_size))
        out = self.fc(out)
        return out

In [38]:
def getData(sticker):
    aapl_df = pd.read_csv('/content/drive/MyDrive/WorkSpace/COMP3027J/data/Stock/Pretest_data/'+sticker+'.csv')
    columns = ['Open','High','Low','Close']
    a_df2 = aapl_df[columns].iloc[1:].values/aapl_df[columns].iloc[:-1] -1
    a_df2 = a_df2*100
    X = shift_concat_df(a_df2, 10)
    X = X.loc[0:len(X)-10]
    X_train = X.loc[0:len(X)-11]
    X_test = X.loc[len(X_train)+4:len(X_train)+4]
    a_df3 = a_df2[['Close']]
    Y_df = shift_concat_df(a_df3.shift(-10),5,2)
    Y_train = Y_df.loc[0:len(X_train)-1]
    Y_train = add_5day_return(Y_train)
    Y_test = Y_df.loc[len(X_train)+4:len(X_train)+4]
    Y_test = add_5day_return(Y_test)
    
    return X_train,Y_train, X_test, Y_test


    

In [60]:
def predict(X_train,Y_train, X_test):
    
    train_set = stock_dataset(X_train,Y_train)
    train_loader = Data.DataLoader(dataset=train_set, batch_size=batch_size)
    
    model = RNN(input_size, hidden_size, num_layers, num_predictions).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        for i, (xs, ys) in enumerate(train_loader):

            xs = xs.to(device)
            ys = ys.to(device)
        
            # Forward pass
            outputs = model(xs.float())
            day_tensor = outputs[0:len(outputs),0:5].reshape(len(outputs),5)
            #print(str(i)+'lallalallal')
            #print(day_tensor)
            #print(day_tensor.size())
            week_tensor = outputs[0:len(outputs),5:6].reshape(len(outputs))
            #print(week_tensor.size())
            day_to_week_tensor = torch.prod(day_tensor*100+1,1)-1
            #print(day_to_week_tensor.size())
            loss1 = criterion(day_to_week_tensor,week_tensor)
            #print(loss1.item())
            loss2 = criterion(outputs, ys.float())
            #print(loss2.item())
            loss = 0.01*loss1+loss2
            #print(loss.item())
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if epoch%10 ==0: 
            print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')## - Test
            
    tt =torch.from_numpy(X_test.values).reshape(len(X_test),10,4)
    tt = tt.to(device)
    outputs = model(tt.float())
    
    return outputs

    

In [64]:
X_train,Y_train, X_test, Y_test = getData('IBM')
outputs = predict(X_train,Y_train, X_test)
print(outputs)
ytt = torch.from_numpy(Y_test.values).float().to(device)
errors = criterion(outputs, ytt)
print(ytt)
print(errors)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/100], Loss: 6.2390
Epoch [11/100], Loss: 4.5604
Epoch [21/100], Loss: 4.4537
Epoch [31/100], Loss: 4.3903
Epoch [41/100], Loss: 4.3575
Epoch [51/100], Loss: 4.3368
Epoch [61/100], Loss: 4.3228
Epoch [71/100], Loss: 4.3127
Epoch [81/100], Loss: 4.3045
Epoch [91/100], Loss: 4.2975
tensor([[-0.0160,  0.0047, -0.0450, -0.0032, -0.0110, -0.0644]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.3518,  3.7861,  3.9013, -1.5813,  0.8140,  6.6173]],
       device='cuda:0')
tensor(12.9667, device='cuda:0', grad_fn=<MseLossBackward>)


In [65]:
X_train,Y_train, X_test, Y_test = getData('BK')
outputs = predict(X_train,Y_train, X_test)
print(outputs)
ytt = torch.from_numpy(Y_test.values).float().to(device)
errors = criterion(outputs, ytt)
print(ytt)
print(errors)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/100], Loss: 1019.8366
Epoch [11/100], Loss: 37.7696
Epoch [21/100], Loss: 18.7730
Epoch [31/100], Loss: 13.6546
Epoch [41/100], Loss: 11.4968
Epoch [51/100], Loss: 10.0732
Epoch [61/100], Loss: 9.0275
Epoch [71/100], Loss: 8.2336
Epoch [81/100], Loss: 7.6226
Epoch [91/100], Loss: 7.1532
tensor([[-0.0153,  0.0695, -0.0182, -0.1054, -0.0142,  0.1152]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.6063, -0.4913,  1.3096, -1.3562,  2.2556,  3.3210]],
       device='cuda:0')
tensor(3.6168, device='cuda:0', grad_fn=<MseLossBackward>)


In [52]:
def do_it():
    fo = open("prediction.csv", "w")
    fo.write('tickers,return_day_1,return_day_2,return_day_3,return_day_4,return_day_5,return_week\n')
    stock_ticker_list = pd.read_csv('/content/drive/MyDrive/WorkSpace/COMP3027J/data/Stock/Stock_Ticker_List.csv')
    stickers = stock_ticker_list['Symbol'].values
    for i in range(len(stickers)):
        X_train,Y_train, X_test, Y_test = getData(stickers[i])
        outputs = predict(X_train,Y_train, X_test)
        print(outputs)
        ytt = torch.from_numpy(Y_test.values).float().to(device)
        print(ytt)
        e = criterion(outputs, ytt)
        print(e.item())
        a = outputs.cpu().detach().view(6).numpy()
        #b = a/100+1
        #print(b)
        #print(np.prod(b))
        #a = np.append(a,np.prod(b)-1)
        s = stickers[i]+','+np.array2string(a,formatter={'float_kind':lambda x: "%.2f" % x},separator=',')[1:-1]
        if i!= (len(stickers)-1):
            s = s + '\n'
        fo.write(s)
        print(stickers[i]+' done!')
    fo.close()
    return None
    

In [55]:
do_it()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/50], Loss: 14184.6611
Epoch [11/50], Loss: 1756.5986
Epoch [21/50], Loss: 1114.1973
Epoch [31/50], Loss: 713.6559
Epoch [41/50], Loss: 321.2790
tensor([[-0.0177, -0.0664, -0.0242,  0.0032, -0.0378, -0.2799]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.5069, -1.2830,  0.2930, -1.1685,  1.8039,  0.1193]],
       device='cuda:0')
1.1300060749053955
AAPL done!
Epoch [1/50], Loss: 528813.4375
Epoch [11/50], Loss: 38813.7188
Epoch [21/50], Loss: 14377.7754
Epoch [31/50], Loss: 7968.7217
Epoch [41/50], Loss: 5216.7446
tensor([[ 0.0342, -0.0209, -0.0005, -0.0120,  0.0854, -0.2140]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.6487,  0.3867,  1.6234, -0.6769,  1.2085,  3.2156]],
       device='cuda:0')
2.774442195892334
ABBV done!
Epoch [1/50], Loss: 535707.2500
Epoch [11/50], Loss: 33851.0547
Epoch [21/50], Loss: 13922.4609
Epoch [31/50], Loss: 6697.1724
Epoch [41/50], Loss: 3775.4065
tensor([[-0.0074,  0.0026, -0.0130,  0.0664,  0.0242, -0.1051]],
 

In [ ]:
stock_ticker_list = pd.read_csv('./data/Stock/Stock_Ticker_List.csv')
stock_ticker_list['Symbol'].values

array(['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'AMGN', 'AMT', 'AMZN',
       'AVGO', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY',
       'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM',
       'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR',
       'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL',
       'GS', 'HD', 'HON', 'IBM'], dtype=object)

In [ ]:
if i != 5:
    print('hello')

hello


In [ ]:
fo = open("foo.txt", "w")
a = outputs.detach().view(5).numpy()
b = a/100+1
a = np.append(a,np.prod(b)-1)
s = np.array2string(a,formatter={'float_kind':lambda x: "%.2f" % x},separator=',')[1:-1]
fo.write(s)
fo.close()

In [ ]:
a = outputs.detach().view(5).numpy()
b = a/100+1
print(b)
print(np.prod(b))
a = np.append(a,np.prod(b)-1)
a

[1.0021372 1.0047051 0.9980838 1.0019469 0.9879443]
0.9947407


array([ 0.21372202,  0.47051078, -0.19162057,  0.19469385, -1.20557022,
       -0.00525928])

In [ ]:
X_train,Y_train, X_test, Y_test = getData('BK')
outputs = predict(X_train,Y_train, X_test)
print(outputs)
ytt = torch.from_numpy(Y_test.values).float()
criterion(outputs, ytt)